In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.types import * 
schema = StructType([
    StructField('order_type', StringType(), False),
    StructField('stop', IntegerType(), True),
    StructField('limit', IntegerType(), True),
    StructField('price', FloatType(), True),
    StructField('stop_price', FloatType(), True),
    StructField('limit_price', FloatType(), True)
])
orders=spark.createDataFrame(data=[],schema=schema)
orders.createOrReplaceTempView("orders")

In [3]:
new_orders=[['sell', 648, 470, 1.2814100000000002, 1.34671, 1.2349100000000002], ['sell', 279, 296, 1.2814100000000002, 1.3098100000000001, 1.25231], ['buy', 819, 963, 1.28241, 1.20001, 1.3782100000000002], ['buy', 148, 613, 1.28241, 1.2671100000000002, 1.34321], ['buy', 425, 726, 1.28241, 1.2394100000000001, 1.35451], ['sell', 708, 663, 1.2814100000000002, 1.35271, 1.21561], ['sell', 996, 753, 1.2814100000000002, 1.38151, 1.2066100000000002], ['sell', 558, 371, 1.2814100000000002, 1.3377100000000002, 1.2448100000000002], ['buy', 595, 749, 1.28241, 1.22241, 1.35681], ['buy', 154, 717, 1.28241, 1.26651, 1.3536100000000002]]

In [4]:
new_orders_df=spark.createDataFrame(new_orders,schema)
new_orders_df

DataFrame[order_type: string, stop: int, limit: int, price: float, stop_price: float, limit_price: float]

In [5]:
orders=orders.union(new_orders_df)
orders.createOrReplaceTempView("orders")
orders.count()

10

In [6]:
price=1.22
display(orders.count())
stop_triggered=spark.sql("\
        SELECT * FROM orders \
        WHERE (order_type='buy' AND {0}<=stop_price)\
            OR (order_type='sell' AND {0}>=stop_price)\
    ".format(price))
limit_triggered=spark.sql("\
        SELECT * FROM orders \
        WHERE (order_type='buy' AND {0}>=limit_price)\
            OR (order_type='sell' AND {0}<=limit_price)\
    ".format(price))
orders=spark.sql("\
        SELECT * FROM orders \
        WHERE ((order_type='buy' AND {0}<limit_price)\
            OR (order_type='sell' AND {0}>limit_price)) \
            AND \
            ((order_type='buy' AND {0}>stop_price)\
            OR (order_type='sell' AND {0}<stop_price))\
    ".format(price))
display(stop_triggered.count())
display(limit_triggered.count())

orders.createOrReplaceTempView("orders")
display(orders.count())

10

4

3

3

In [7]:
orders=spark.sql("\
        SELECT * FROM orders \
        WHERE (order_type='buy' AND {0}<limit_price)\
            OR (order_type='sell' AND {0}>limit_price)\
            OR (order_type='buy' AND {0}>stop_price)\
            OR (order_type='sell' AND {0}<stop_price)\
    ".format(price))
orders.summary()

DataFrame[summary: string, order_type: string, stop: string, limit: string, price: string, stop_price: string, limit_price: string]

In [8]:
result=spark.sql("SELECT count(*) FROM orders")
result.show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [9]:
orders.count()

3

In [15]:
limit_triggered.show()

+----------+----+-----+-------+----------+-----------+
|order_type|stop|limit|  price|stop_price|limit_price|
+----------+----+-----+-------+----------+-----------+
|      sell| 648|  470|1.28141|   1.34671|    1.23491|
|      sell| 279|  296|1.28141|   1.30981|    1.25231|
|      sell| 558|  371|1.28141|   1.33771|    1.24481|
+----------+----+-----+-------+----------+-----------+



In [47]:
list(map(lambda row:row.asDict(), limit_triggered.collect()))


[{'order_type': 'sell',
  'stop': 648,
  'limit': 470,
  'price': 1.2814099788665771,
  'stop_price': 1.346709966659546,
  'limit_price': 1.234910011291504},
 {'order_type': 'sell',
  'stop': 279,
  'limit': 296,
  'price': 1.2814099788665771,
  'stop_price': 1.3098100423812866,
  'limit_price': 1.252310037612915},
 {'order_type': 'sell',
  'stop': 558,
  'limit': 371,
  'price': 1.2814099788665771,
  'stop_price': 1.3377100229263306,
  'limit_price': 1.2448099851608276}]

In [31]:
result